<a href="https://colab.research.google.com/github/blanco-herrero/Interviews/blob/main/David_TOPIC_MODELLING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
# Importing modules
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim import corpora
from gensim import models
from gensim import matutils
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
os.chdir('..')
from glob import glob
from string import punctuation
import nltk
from nltk.sentiment import vader
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
# Upload the file
from google.colab import files
files.upload()

Saving Entrevistas.txt to Entrevistas (2).txt


{'Entrevistas.txt': b"\xef\xbb\xbfS\xc3\xad, claro. Yo creo que est\xc3\xa1n en el d\xc3\xada a d\xc3\xada, evidentemente. La verdad que siempre se buscan m\xc3\xa1s chivos expiatorios. En este caso, quiz\xc3\xa1 de forma no tan exacerbada, en este caso Espa\xc3\xb1a es un pa\xc3\xads curioso. Lo cual no quiere decir que no haya racismo en Espa\xc3\xb1a, creo que lo hay, lo que pasa que lo disimulamos m\xc3\xa1s o lo disfrazamos. Pero s\xc3\xad, por supuesto, ha habido y lo hay. Adem\xc3\xa1s que aqu\xc3\xad, si rascas un poco yo creo que aparece. Y bueno, hablas del tema del lenguaje, me parece el tema fundamental porque es lo que, de alguna manera, traduce precisamente esa especie de pensamiento subterr\xc3\xa1neo, y en la terminolog\xc3\xada respecto al otro, pues cuando se habla de invasi\xc3\xb3n, o se habla de avalancha, hay una terminolog\xc3\xada que parece completamente inocua, y no lo es en absoluto, porque hay connotaciones ah\xc3\xad b\xc3\xa9licas tambi\xc3\xa9n, o de amen

In [79]:
# Read data into an object
entrevistas_df = pd.read_csv("Entrevistas.txt", header= None, sep='\t')
entrevistas = entrevistas_df.T.squeeze()

In [80]:
# Print out the first rows of the file
entrevistas

0      Sí, claro. Yo creo que están en el día a día, ...
1      Bueno, tendría que hacer memoria. La verdad es...
2      Bueno, la verdad es que cuando dedicas espacio...
3      La verdad es que hay, esto tiene que ver con l...
4      Bueno, los que buscan enemigos, los que simpli...
                             ...                        
519    Honestamente, es un artículo que espero que le...
520    Con las estadísticas, por ejemplo, me he hecho...
521    Sí, escuchar, escuchar, escuchar. Como decían ...
522    La movida gore, que yo no la meto normalmente ...
523                                                   21
Name: 0, Length: 524, dtype: object

In [81]:
# Remove the columns (if the case)
# trans = trans.drop(columns=['id', 'location', 'language'], axis=1).sample(100)

In [82]:
import re
(lambda x: re.sub('[-–_–,:/.¡!¿?...]','', x))

<function __main__.<lambda>(x)>

In [83]:
# Remove punctuation
entrevistas = entrevistas.map(lambda x: re.sub('[-–_–,:/.¡!¿?...]','', x))
entrevistas

0      Sí claro Yo creo que están en el día a día evi...
1      Bueno tendría que hacer memoria La verdad es q...
2      Bueno la verdad es que cuando dedicas espacio ...
3      La verdad es que hay esto tiene que ver con la...
4      Bueno los que buscan enemigos los que simplifi...
                             ...                        
519    Honestamente es un artículo que espero que lea...
520    Con las estadísticas por ejemplo me he hecho c...
521    Sí escuchar escuchar escuchar Como decían Faem...
522    La movida gore que yo no la meto normalmente e...
523                                                   21
Name: 0, Length: 524, dtype: object

In [84]:
# Convert uppercase to lowercase
entrevistas.map(lambda x: x.lower())

0      sí claro yo creo que están en el día a día evi...
1      bueno tendría que hacer memoria la verdad es q...
2      bueno la verdad es que cuando dedicas espacio ...
3      la verdad es que hay esto tiene que ver con la...
4      bueno los que buscan enemigos los que simplifi...
                             ...                        
519    honestamente es un artículo que espero que lea...
520    con las estadísticas por ejemplo me he hecho c...
521    sí escuchar escuchar escuchar como decían faem...
522    la movida gore que yo no la meto normalmente e...
523                                                   21
Name: 0, Length: 524, dtype: object

In [85]:
# PREPARE DATA FOR LDA ANALYSIS

# Generate the stop words
stop_words = stopwords.words('spanish')

stop_words.extend(['http', 'https', 'tco', 'httpstco', 'co', 'pa', 'si', 
                   'rt', 'etc', 'ect', 'tco', 'xa0', 't', 'si', 'q', 'd', 'lo', 'ell', 'call', 
                   'pkly', 'onde', 'pese', 'tb', '000', 'dos', 'tres', 'cia', 'vez', 'ves', 
                   'mira', 'tan', 'tal', 'dar', 'da', 'das', 'dan', 'uma', 'va', 'van', 'ser',
                   'ahí', 'ahi', 'tras', 'detrás', 'detras', 'creo', 'hecho', 'ejemplo', 'demas', 
                   'demás', 'ademas', 'además', 'resto', 'pensar', 'sino', 'decir', 'lado', 
                   'parece', 'piensa', 'pase', 'pesar', 'entender', 'alguien', 'dicho', 'supuesto', 
                   'alli', 'allí', 'aun', 'cualquier', 'cuestión', 'contenido', 'contenidos', 
                   'simplemente', 'símplemente', 'habia', 'parte', 'acaso', 'biden', 'toda', 'todas',
                   'gracias', 'despues', 'después', 'of', 'and', 'his', 'the', 'cada', 'to', 
                   'in', 'ello', 'quiere', 'buenas', 'ningún', 'día', 'pocos', 'cómo', 'como', 'puede', 
                   'pone', 'mientras', 'garcimoreno', 'cabrita', 'veintimillapier', 'qls', 'colocolo', 
                   'gyzytqqtbm', 'sólo', 'solo', 'ahora', 'frente', 'hacen', 'hace', 'bien', 'años', 
                   'nueva', 'luego', 'así', 'asi','claro', 'ver', 'debería', 'video', 'estan', 'mas', 
                   'menos', 'hacer', 'solo', 'pues', 'incluso', 'meses', 'vale', 'dia', 'evidentemente', 'tambien', 'entonces',
                   'bueno', 'gente', 'discurso', 'aqui', 'veces', 'digo', 'vamos', 'momento', 'forma', 'cosas', 'tipo'])

len(stop_words)

452

In [86]:
stop_words

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [87]:
#spanish_stopwords.append('')
#spanish_stopwords.remove('')

In [88]:
def sent_to_words(sentences):
    for sentence in sentences:
        
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]  

In [89]:
data = entrevistas.values.tolist()
data_words = list(sent_to_words(data))

In [90]:
# remove stop words
data_words = remove_stopwords(data_words)

In [91]:
print(data_words[:1][0][:30])

['verdad', 'siempre', 'buscan', 'chivos', 'expiatorios', 'caso', 'quiza', 'forma', 'exacerbada', 'caso', 'espana', 'pais', 'curioso', 'racismo', 'espana', 'pasa', 'disimulamos', 'disfrazamos', 'aqui', 'rascas', 'aparece', 'hablas', 'tema', 'lenguaje', 'tema', 'fundamental', 'alguna', 'manera', 'traduce', 'precisamente']


In [92]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [93]:
# LDA model training

from pprint import pprint
# Number of topics
num_topics = 4

# Build LDA model
lda_model = models.ldamodel.LdaModel(num_topics=num_topics, corpus=corpus, id2word=id2word, random_state=None, 
                                     per_word_topics=True, alpha='auto', passes=2)
lda_model.print_topics(num_words=10)

[(0,
  '0.006*"aqui" + 0.006*"verdad" + 0.005*"personas" + 0.004*"veces" + 0.004*"medios" + 0.004*"odio" + 0.003*"mejor" + 0.003*"momento" + 0.003*"digo" + 0.003*"vamos"'),
 (1,
  '0.007*"odio" + 0.005*"muchas" + 0.005*"personas" + 0.005*"cosas" + 0.005*"veces" + 0.004*"digo" + 0.004*"tema" + 0.004*"aqui" + 0.004*"persona" + 0.004*"migrantes"'),
 (2,
  '0.008*"veces" + 0.007*"informacion" + 0.006*"siempre" + 0.005*"odio" + 0.005*"cosas" + 0.004*"trabajo" + 0.004*"espana" + 0.004*"persona" + 0.004*"muchas" + 0.004*"verdad"'),
 (3,
  '0.010*"odio" + 0.007*"tema" + 0.006*"informacion" + 0.005*"siempre" + 0.005*"personas" + 0.005*"final" + 0.004*"aqui" + 0.004*"forma" + 0.004*"medios" + 0.004*"cosas"')]

In [94]:
# Print the keywords of the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
lda_model.print_topics(num_words=40)


[(0,
  '0.006*"aqui" + 0.006*"verdad" + 0.005*"personas" + 0.004*"veces" + '
  '0.004*"medios" + 0.004*"odio" + 0.003*"mejor" + 0.003*"momento" + '
  '0.003*"digo" + 0.003*"vamos"'),
 (1,
  '0.007*"odio" + 0.005*"muchas" + 0.005*"personas" + 0.005*"cosas" + '
  '0.005*"veces" + 0.004*"digo" + 0.004*"tema" + 0.004*"aqui" + '
  '0.004*"persona" + 0.004*"migrantes"'),
 (2,
  '0.008*"veces" + 0.007*"informacion" + 0.006*"siempre" + 0.005*"odio" + '
  '0.005*"cosas" + 0.004*"trabajo" + 0.004*"espana" + 0.004*"persona" + '
  '0.004*"muchas" + 0.004*"verdad"'),
 (3,
  '0.010*"odio" + 0.007*"tema" + 0.006*"informacion" + 0.005*"siempre" + '
  '0.005*"personas" + 0.005*"final" + 0.004*"aqui" + 0.004*"forma" + '
  '0.004*"medios" + 0.004*"cosas"')]


[(0,
  '0.006*"aqui" + 0.006*"verdad" + 0.005*"personas" + 0.004*"veces" + 0.004*"medios" + 0.004*"odio" + 0.003*"mejor" + 0.003*"momento" + 0.003*"digo" + 0.003*"vamos" + 0.003*"tema" + 0.003*"cosas" + 0.003*"hablar" + 0.003*"pais" + 0.003*"manera" + 0.003*"persona" + 0.003*"periodismo" + 0.002*"espana" + 0.002*"dice" + 0.002*"migrantes" + 0.002*"trabajo" + 0.002*"informacion" + 0.002*"mucha" + 0.002*"tener" + 0.002*"datos" + 0.002*"final" + 0.002*"anos" + 0.002*"tipo" + 0.002*"cosa" + 0.002*"muchas" + 0.002*"puedes" + 0.002*"siempre" + 0.002*"tiempo" + 0.002*"nadie" + 0.002*"mundo" + 0.002*"comunicacion" + 0.002*"companeros" + 0.002*"vida" + 0.002*"mismo" + 0.002*"intento"'),
 (1,
  '0.007*"odio" + 0.005*"muchas" + 0.005*"personas" + 0.005*"cosas" + 0.005*"veces" + 0.004*"digo" + 0.004*"tema" + 0.004*"aqui" + 0.004*"persona" + 0.004*"migrantes" + 0.003*"tener" + 0.003*"espana" + 0.003*"redes" + 0.003*"final" + 0.003*"sociales" + 0.003*"mismo" + 0.003*"trabajo" + 0.003*"siempre" + 0.0

In [95]:
# Analyze LDA model results
!pip install pyLDAvis
!python -m pip install -U pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models
import pickle 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))

In [97]:
# This is a bit time consuming - make the if statement True
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [98]:
# Load the prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.019081  0.034331       1        1  35.972082
2     -0.024219 -0.029953       2        1  26.513836
1      0.004862 -0.002916       3        1  23.574290
0      0.038439 -0.001462       4        1  13.939792, topic_info=            Term        Freq       Total Category  logprob  loglift
5           aqui  224.000000  224.000000  Default  30.0000  30.0000
118       verdad  197.000000  197.000000  Default  29.0000  29.0000
116        veces  283.000000  283.000000  Default  28.0000  28.0000
299       medios  163.000000  163.000000  Default  27.0000  27.0000
603         digo  170.000000  170.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
414         tipo   16.756161  137.166272   Topic4  -6.1611  -0.1320
108        tener   17.342732  162.759025   Topic4  -6.1266  -0.2687
894  informacion   18.535929  272.888944   Topic4  -6.0601  -0.7189
233        final   16.781211  207.058944   Topic4  -6.1596  -0.5423
74        muchas   16.510214  201.109581   Topic4  -6.1758  -0.5294

[359 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
10477      1  0.151167  acogemos
10477      3  0.755836  acogemos
949        1  0.497317   acuerdo
949        2  0.188637   acuerdo
949        3  0.188637   acuerdo
...      ...       ...       ...
824        1  0.461138       voy
824        2  0.188648       voy
824        3  0.282971       voy
824        4  0.073363       voy
2550       4  0.664889        xv

[687 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

In [ ]:
# Take a look to the texts and their topics
topics = pd.DataFrame([dict(lda_model.get_document_topics(doc, minimum_probability=0.1))
                      for doc in corpus])
meta = entrevistas.iloc[entrevistas.index].drop(columns=[0]).reset_index(drop=True)
tpd = pd.concat([meta, topics], axis=1)
tpd [:50]